# Evaluación de agente respuestas

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os


In [2]:

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

# Evaluacion

In [8]:
import pandas as pd
from dotenv import load_dotenv
from langchain.evaluation.qa import QAEvalChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Cargar .env
load_dotenv()

# Cargar dataset
df = pd.read_csv(r"E:\Maestría_Eafit2024\Maestria\Semestre_3\NLP\custom_agent-main\custom_agent-main\Evaluacion\eval_dataset.csv")

# LLM para evaluación
llm_evaluacion = ChatOpenAI(model="o4-mini", temperature=1)

# LLM para simular respuestas de estudiantes
llm_respuestas = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)

# Prompt de estudiante
student_prompt = PromptTemplate.from_template(
    """Imagina que eres un estudiante que responde una evaluación de estadística.
A veces respondes bien y otras veces te equivocas o das respuestas incompletas.

Pregunta: {pregunta}

Responde en una o dos frases, como un estudiante humano. No siempre respondas perfectamente."""
)

chain_respuestas = LLMChain(llm=llm_respuestas, prompt=student_prompt)

# Crear la cadena de evaluación
eval_chain = QAEvalChain.from_llm(
    llm=llm_evaluacion,
    criteria=["correctness"],
    prompt=PromptTemplate.from_template(
        "Dada la pregunta: {query}\n"
        "La respuesta del estudiante: {result}\n"
        "La respuesta correcta esperada: {answer}\n\n"
        "¿La respuesta es correcta? Responde sólo 'Sí' o 'No', y explica en una línea por qué."
    )
)

# Preparar ejemplos
examples = df.to_dict(orient="records")

# Evaluar
results = []
respuestas_generadas = []

for example in examples:
    # Generar respuesta de estudiante
    respuesta = chain_respuestas.invoke({"pregunta": example["query"]})
    respuesta_estudiante = respuesta["text"]

    respuestas_generadas.append(respuesta_estudiante)

    # Evaluar
    result = eval_chain.evaluate_strings(
        prediction=respuesta_estudiante,
        input=example["query"],
        reference=example["answer"],
    )
    results.append(result)

# Agregar las respuestas generadas y evaluaciones al dataframe
df["respuesta_estudiante"] = respuestas_generadas
df["evaluacion"] = [r["reasoning"] for r in results]

# Mostrar resultados
print(df)

# Guardar
df.to_csv("resultados_evaluacion_simulada.csv", index=False)




                                                query  \
0      ¿Qué mide la varianza en un conjunto de datos?   
1            ¿Qué representa una probabilidad de 0.8?   
2            ¿Qué es un evento mutuamente excluyente?   
3                     ¿Qué es una variable aleatoria?   
4       ¿Qué indica la media de un conjunto de datos?   
5   ¿Qué diferencia hay entre una población y una ...   
6                        ¿Qué es un espacio muestral?   
7   ¿Cuál es el propósito de un intervalo de confi...   
8   ¿Qué significa que dos eventos sean independie...   
9                   ¿Qué es un experimento aleatorio?   
10              ¿Qué describe la desviación estándar?   
11                 ¿Qué es una distribución binomial?   
12     ¿Qué se entiende por probabilidad condicional?   
13                 ¿Qué es un diagrama de dispersión?   
14  ¿Qué representa un coeficiente de correlación ...   
15  ¿Qué es un error tipo I en una prueba de hipót...   
16  ¿Qué es un error tipo II en

In [ ]:
##Evaluacion con errores

import random
import pandas as pd
from dotenv import load_dotenv
from langchain.evaluation.qa import QAEvalChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Cargar .env
load_dotenv()

# Cargar dataset
df = pd.read_csv(r"E:\Maestría_Eafit2024\Maestria\Semestre_3\NLP\custom_agent-main\custom_agent-main\Evaluacion\eval_dataset.csv")

# LLM para evaluación
llm_evaluacion = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# LLM para simular respuestas de estudiantes
llm_respuestas = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9)

# Prompt de estudiante
student_prompt_correcta = PromptTemplate.from_template(
    """Imagina que eres un estudiante que responde una evaluación de estadística. 
    Responde correctamente la siguiente pregunta:

    Pregunta: {pregunta}

    Responde de forma precisa y completa como un estudiante que sabe la respuesta."""
)

student_prompt_incorrecta = PromptTemplate.from_template(
    """Imagina que eres un estudiante que responde una evaluación de estadística. 
    A veces te equivocas o no sabes la respuesta correctamente. 
    Responde de forma incorrecta o incompleta a la siguiente pregunta:

    Pregunta: {pregunta}

    Responde con algo que parece lógico, pero es incorrecto."""
)

chain_respuestas_correctas = LLMChain(llm=llm_respuestas, prompt=student_prompt_correcta)
chain_respuestas_incorrectas = LLMChain(llm=llm_respuestas, prompt=student_prompt_incorrecta)

# Crear la cadena de evaluación
eval_chain = QAEvalChain.from_llm(
    llm=llm_evaluacion,
    criteria=["correctness"],
    prompt=PromptTemplate.from_template(
        "Dada la pregunta: {query}\n"
        "La respuesta del estudiante: {result}\n"
        "La respuesta correcta esperada: {answer}\n\n"
        "¿La respuesta es correcta? Responde sólo 'Sí' o 'No', y explica en una línea por qué."
    )
)

# Porcentaje de error (30% de respuestas incorrectas)
porcentaje_error = 0.3  # 30% de respuestas incorrectas

# Preparar ejemplos
examples = df.to_dict(orient="records")

# Evaluar
results = []
respuestas_generadas = []

for example in examples:
    # Decidir si la respuesta será correcta o incorrecta
    if random.random() < porcentaje_error:
        # Generar respuesta incorrecta
        respuesta = chain_respuestas_incorrectas.invoke({"pregunta": example["query"]})
    else:
        # Generar respuesta correcta
        respuesta = chain_respuestas_correctas.invoke({"pregunta": example["query"]})

    respuesta_estudiante = respuesta["text"]
    respuestas_generadas.append(respuesta_estudiante)

    # Evaluar
    result = eval_chain.evaluate_strings(
        prediction=respuesta_estudiante,
        input=example["query"],
        reference=example["answer"],
    )
    results.append(result)

# Agregar las respuestas generadas y evaluaciones al dataframe
df["respuesta_estudiante"] = respuestas_generadas
df["evaluacion"] = [r["reasoning"] for r in results]

# Mostrar resultados
print(df)

# Guardar
df.to_csv("resultados_evaluacion_simulada.csv", index=False)


                                                query  \
0      ¿Qué mide la varianza en un conjunto de datos?   
1            ¿Qué representa una probabilidad de 0.8?   
2            ¿Qué es un evento mutuamente excluyente?   
3                     ¿Qué es una variable aleatoria?   
4       ¿Qué indica la media de un conjunto de datos?   
5   ¿Qué diferencia hay entre una población y una ...   
6                        ¿Qué es un espacio muestral?   
7   ¿Cuál es el propósito de un intervalo de confi...   
8   ¿Qué significa que dos eventos sean independie...   
9                   ¿Qué es un experimento aleatorio?   
10              ¿Qué describe la desviación estándar?   
11                 ¿Qué es una distribución binomial?   
12     ¿Qué se entiende por probabilidad condicional?   
13                 ¿Qué es un diagrama de dispersión?   
14  ¿Qué representa un coeficiente de correlación ...   
15  ¿Qué es un error tipo I en una prueba de hipót...   
16  ¿Qué es un error tipo II en

# Evaluación de agente feedback

In [29]:
import pandas as pd
from dotenv import load_dotenv
from langchain.evaluation.qa import QAEvalChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Cargar .env
load_dotenv()

# Cargar dataset de feedback
df = pd.read_csv(r"E:\Maestría_Eafit2024\Maestria\Semestre_3\NLP\custom_agent-main\custom_agent-main\Evaluacion\feedback_dataset.csv")

# LLM para evaluación
llm_evaluacion = ChatOpenAI(model="gpt-3.5-turbo", temperature=1)

# LLM para generar respuestas
llm_respuestas = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)



In [33]:
from langchain.prompts import PromptTemplate

# Ajuste del PromptTemplate para que acepte 'query', 'result' y 'answer'
feedback_prompt = PromptTemplate.from_template(
    """Dada la respuesta del estudiante: {query}
    La retroalimentación generada por el modelo: {result}
    La retroalimentación real: {answer}

    ¿La retroalimentación generada por el modelo es adecuada? Responde sólo 'Sí' o 'No', y explica en una línea por qué.
    """
)

# Crear la cadena de evaluación (QAEvalChain) con las tres variables
eval_chain = QAEvalChain.from_llm(
    llm=llm_evaluacion,
    criteria=["correctness"],
    prompt=feedback_prompt
)

# Preparar ejemplos
examples = df.to_dict(orient="records")

# Evaluar
results = []
feedback_generado = []

for example in examples:
    # Generar feedback del modelo
    feedback = chain_feedback.invoke({"query": example["query"], "answer": example["answer"]})
    feedback_text = feedback["text"]

    feedback_generado.append(feedback_text)

    # Evaluar si el feedback generado es adecuado
    result = eval_chain.evaluate_strings(
        prediction=feedback_text,        # Feedback generado por el modelo (prediction)
        input=example["query"],          # Respuesta del estudiante (input)
        reference=example["answer"],     # Retroalimentación real proporcionada en el dataset (reference)
    )
    results.append(result)

# Agregar las respuestas generadas y evaluaciones al dataframe
df["feedback_generado"] = feedback_generado
df["evaluacion_feedback"] = [r["reasoning"] for r in results]

# Mostrar resultados
print(df)

# Guardar
df.to_csv("resultados_evaluacion_feedback.csv", index=False)


                                             pregunta  \
0      ¿Qué mide la varianza en un conjunto de datos?   
1            ¿Qué representa una probabilidad de 0.8?   
2            ¿Qué es un evento mutuamente excluyente?   
3                     ¿Qué es una variable aleatoria?   
4       ¿Qué indica la media de un conjunto de datos?   
5   ¿Qué diferencia hay entre una población y una ...   
6                        ¿Qué es un espacio muestral?   
7   ¿Cuál es el propósito de un intervalo de confi...   
8   ¿Qué significa que dos eventos sean independie...   
9                   ¿Qué es un experimento aleatorio?   
10              ¿Qué describe la desviación estándar?   
11                 ¿Qué es una distribución binomial?   
12     ¿Qué se entiende por probabilidad condicional?   
13                 ¿Qué es un diagrama de dispersión?   
14  ¿Qué representa un coeficiente de correlación ...   
15  ¿Qué es un error tipo I en una prueba de hipót...   
16  ¿Qué es un error tipo II en